In [1]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 
!pip install pytorch-lightning 
# !pip install wandb
%pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 6.5 MB/s 
     |████████████████████████████████| 1.3 MB 80.8 MB/s 
     |████████████████████████████████| 120 kB 87.2 MB/s 
     |████████████████████████████████| 880 kB 75.3 MB/s 
     |████████████████████████████████| 3.3 MB 58.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=cd25a0d318175ae053a0e219cb59705c72278b0f47836b83bdfedc04c278a1d9
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 306 kB 9.9 MB/s 
     |████████████████████████████████| 212 kB 73.8 MB/s 
     |████████████████████████████████| 115 kB 89.4 MB/s 
Looking in indexes: https://pypi.org/simple

In [2]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from transformers import (
    MT5ForConditionalGeneration,
    MT5TokenizerFast,
)
from transformers import AutoTokenizer, BertForTokenClassification
import pandas as pd
import wandb
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import torch
import pickle
from tqdm import tqdm
import os
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from typing import Optional

In [5]:
df = pickle.load(open('drive/MyDrive/AIBuilders/tpth/mlm_ds_40k_nova.pkl', 'rb'))

In [6]:
df

,input_ids,attention_mask,labels
0,"[5, 10, 9337, 10, 21993, 688, 2902, 874, 350, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 9337, 10, 21993, 688, 2902, 874, 350, ..."
1,"[5, 10, 2004, 10, 38, 293, 15809, 4076, 10, 14...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 2004, 10, 38, 293, 15809, 4076, 10, 14..."
2,"[5, 984, 23, 545, 1223, 6419, 10, 3133, 5033, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 984, 23, 545, 1223, 6419, 10, 3133, 5033, ..."
3,"[5, 10, 38, 5242, 869, 17440, 10, 25004, 2004,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 38, 5242, 869, 17440, 10, 357, 2004, 2..."
4,"[5, 10, 11617, 12071, 27, 10478, 221, 2405, 34...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 11617, 12071, 27, 10478, 221, 2405, 34..."
...,...,...,...
42885,"[5, 10, 10508, 102, 32729, 2306, 15933, 368, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 10508, 102, 32729, 2306, 15933, 368, 1..."
42886,"[5, 10, 1417, 26, 10980, 36, 491, 612, 21, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 1417, 26, 10980, 36, 491, 612, 21, 200..."
42887,"[5, 10, 25004, 25004, 25004, 25004, 10, 2004, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 6850, 1448, 265, 265, 10, 2004, 10, 22..."
42888,"[5, 13276, 5948, 320, 1259, 88, 154, 5364, 33,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 13276, 5948, 320, 1259, 88, 154, 5364, 33,..."


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', model_max_length=512)
tokenizer_data = pickle.load(open('drive/MyDrive/AIBuilders/tpth/tokenizer_40k_nova.pkl', 'rb'))

In [ ]:
a = len(df)
blacklist = ['<s>', '</s>', '<pad>']
for i in tqdm(range(a)):
  text = tokenizer_data.convert_ids_to_tokens(df.iloc[i]['input_ids'])
  text = [j for j in text if j not in blacklist]

  labels = tokenizer_data.convert_ids_to_tokens(df.iloc[i]['labels'])
  labels = [j for j in labels if j not in blacklist]
  # print(len(text), len(labels))
  if len(text) != len(labels):
    print("ASS")
    break

100%|██████████| 42890/42890 [00:32<00:00, 1321.46it/s]


In [ ]:
df_test = df[37890:]
df = df[:37890]
df_train, df_val = np.split(df.sample(frac=1, random_state=42),
                            [int(.9 * len(df))])

In [ ]:
#make dataset for mt5
def convert_to_text_no_split_mask(data):
  len_data = len(data)
  blacklist = ['<s>', '</s>', '<pad>']
  output_df = pd.DataFrame(columns=['source_text', 'target_text'])
  for i in tqdm(range(len_data)):
    text = tokenizer_data.convert_ids_to_tokens(data.iloc[i]['input_ids'])
    text = [j for j in text if j not in blacklist]
    len_text = len(text)
    for j in range(len_text):
      text[j] = text[j].replace("▁", " ")
      text[j] = text[j].replace("_", " ")
    text = "แก้คำผิด: " + "".join(text)
    text = text.strip()

    labels = tokenizer_data.convert_ids_to_tokens(data.iloc[i]['labels'])
    labels = [j for j in labels if j not in blacklist]
    len_labels = len(labels)
    for j in range(len_labels):
      labels[j] = labels[j].replace("▁", " ")
      labels[j] = labels[j].replace("_", " ")
    input_labels = "".join(labels)
    input_labels = input_labels.strip()

    # print(f"TEXT: {text}")
    # print(f"LABELS: {labels}")
    row = pd.Series([text, input_labels], index=output_df.columns)
    output_df = output_df.append(row, ignore_index=True)

    # row = pd.Series([text, labels], index=output_df.columns)
    # output_df = output_df.append(row, ignore_index=True)
  return output_df

In [ ]:
test_no_split_df = convert_to_text_no_split_mask(df_test)

100%|██████████| 5000/5000 [00:12<00:00, 406.05it/s]


In [ ]:
pickle.dump(test_no_split_df, open('test_mt5_no_split_tpth_40k.pkl', 'wb'))

In [ ]:
test_no_split_df

,source_text,target_text
0,แก้คำผิด: จาก ที่ดูvoovของน้อง<mask>คือน้องออ...,จาก ที่ดูvoovของน้อง แล้วคือน้องออกมาแก้ตัวให้...
1,แก้คำผิด: ขอพูดถึงสิค<mask> จาก สเตจวันเกิดขอ...,ขอพูดถึงสิคนิดหนึ่ ง จาก สเตจวันเกิดของน้อง ที...
2,แก้คำผิด: f4 2018 ฉาก จะดีก็ดี<mask><mask><ma...,f4 2018 ฉาก จะดีก็ดีปะล้ําปะเหลือ ฉาก จะสุข...
3,แก้คำผิด: เห็นมีคนรี<mask>มารีวิวเพิ่ม เราอ่า...,เห็นมีคนรีเรื่อย ๆมารีวิวเพิ่ม เราอ่านจบแล้วจ้...
4,แก้คำผิด: คุณโอ๊ตเหมือนเรา<mask><mask> <mask>...,คุณโอ๊ตเหมือนเราเลย ฮื อ คือเคยแอบไม่ชอบพี่...
...,...,...
4995,แก้คำผิด: เนื่องจาก น้องเคทจะมีสอบสั มภาษณ์เข...,เนื่องจาก น้องเคทจะมีสอบสั มภาษณ์เข้าเรียนมหาว...
4996,แก้คำผิด: กลับบ้านมาพ่อกับแม่ก็ถามปกติให้รีวิ...,กลับบ้านมาพ่อกับแม่ก็ถามปกติให้รีวิวข้อสอบให้ ...
4997,แก้คำผิด: <mask><mask><mask><mask> รีวิว stor...,อีดอก รีวิว storm rising ด่วนค่ะ (แบบพยายา ม...
4998,แก้คำผิด: ระบบakb48เราลงสุ่มกับโอชิที่อยา กถ่...,ระบบakb48เราลงสุ่มกับโอชิที่อยา กถ่ายถ้าสุ่มได...


In [ ]:
val_no_split_df = convert_to_text_no_split_mask(df_val)

100%|██████████| 3789/3789 [00:09<00:00, 387.90it/s]


In [ ]:
pickle.dump(val_no_split_df, open('val_mt5_no_split_tpth_40k.pkl', 'wb'))

In [ ]:
val_no_split_df

,source_text,target_text
0,แก้คำผิด: แค่พูดว่ารักชาติ ศาสน์ กษัตริย์ไม่ไ...,แค่พูดว่ารักชาติ ศาสน์ กษัตริย์ไม่ได้แปลว่าคุณ...
1,แก้คำผิด: อย่างแรกเลยที่ชอบคือเรื่องนี้เป็นฟี...,อย่างแรกเลยที่ชอบคือเรื่องนี้เป็นฟีลกู้ด ที่น่...
2,แก้คำผิด: รีวิวมาม่าอุ่นร้อนอัตโนมัติของจีน ล...,รีวิวมาม่าอุ่นร้อนอัตโนมัติของจีน ลืมถ่ายตอนเอ...
3,แก้คำผิด: กลับบ้าน<mask><mask>แม่ก็จะฟ้อง<mas...,กลับบ้านครั้งหนึ่ง แม่ก็จะฟ้อง แล้ว พ่อก็จะเถี...
4,แก้คำผิด: เธอ คือคนเดียวที่ทําให้เรามีกําลังใ...,เธอ คือคนเดียวที่ทําให้เรามีกําลังใจในทุกๆวันแ...
...,...,...
3784,แก้คำผิด: **มึ งเห็นผู้หญิงที่ได้แค่เป็นตัวเล...,**มึ งเห็นผู้หญิงที่ได้แค่เป็นตัวเลือกทั้งสองต...
3785,แก้คำผิด: จริงๆ นะ ผู้หญิงสมัยนี้แย่มาก แทนที...,จริงๆ นะ ผู้หญิงสมัยนี้แย่มาก แทนที่จะปกป้องผู...
3786,แก้คำผิด: -หัวเขียว=พริ้ว<mask>มันมีความคล้าย...,-หัวเขียว=พริ้วพริ้ว<mspมันมีความคล้ายนะเออเป็...
3787,แก้คำผิด: สงสารพ่อแม่....เลี้ยงไง<mask><mask>...,สงสารพ่อแม่....เลี้ยงไงเนี่ย ให้ลูกด่าคนอื่นไม...


In [ ]:
#make dataset for mt5
def convert_to_text(data):
  len_data = len(data)
  blacklist = ['<s>', '</s>', '<pad>']
  output_df = pd.DataFrame(columns=['source_text', 'target_text'])
  for i in tqdm(range(len_data)):
    text = tokenizer_data.convert_ids_to_tokens(data.iloc[i]['input_ids'])
    text = [j for j in text if j not in blacklist]
    len_text = len(text)
    for j in range(len_text):
      text[j] = text[j].replace("▁", " ")
      text[j] = text[j].replace("_", " ")
    # text = "แก้คำผิด: " + "".join(text)
    # text = text.strip()

    labels = tokenizer_data.convert_ids_to_tokens(data.iloc[i]['labels'])
    labels = [j for j in labels if j not in blacklist]
    len_labels = len(labels)
    for j in range(len_labels):
      labels[j] = labels[j].replace("▁", " ")
      labels[j] = labels[j].replace("_", " ")
    input_labels = "".join(labels)
    input_labels = input_labels.strip()

    # print(f"TEXT: {text}")
    # print(f"LABELS: {labels}")
    for j in range(len_text):
      if text[j] == "<mask>":
        input_text = labels.copy()
        input_text[j] = "<mask>"
        # print(f"INPUT TEXT: {input_text}")
        input_text = "แก้คำผิด: " + "".join(input_text)
        input_text = input_text.strip()
        row = pd.Series([input_text, input_labels], index=output_df.columns)
        output_df = output_df.append(row, ignore_index=True)

    # row = pd.Series([text, labels], index=output_df.columns)
    # output_df = output_df.append(row, ignore_index=True)
  return output_df

In [ ]:
train_df = convert_to_text(df_train)
valid_df = convert_to_text(df_val)
test_df = convert_to_text(df_test)

100%|██████████| 5000/5000 [01:02<00:00, 80.30it/s]


In [ ]:
pickle.dump(train_df, open('train_mt5_tpth_40k.pkl', 'wb'))
pickle.dump(valid_df, open('valid_mt5_tpth_40k.pkl', 'wb'))
pickle.dump(test_df, open('test_mt5_tpth_40k.pkl', 'wb'))

In [ ]:
train_df = pickle.load(open('drive/MyDrive/AIBuilders/tpth/train_mt5_tpth_40k.pkl', 'rb'))
valid_df = pickle.load(open('drive/MyDrive/AIBuilders/tpth/valid_mt5_tpth_40k.pkl', 'rb'))
test_df = pickle.load(open('drive/MyDrive/AIBuilders/tpth/test_mt5_tpth_40k.pkl', 'rb'))

In [ ]:
train_df

,source_text,target_text
0,แก้คำผิด: ส่วนวันไหนถ้าเกิดอยา กกินbuffetเราจ...,ส่วนวันไหนถ้าเกิดอยา กกินbuffetเราจะไปกินตอนปร...
1,แก้คำผิด: ส่วนวันไหนถ้าเกิดอยา กกินbuffetเราจ...,ส่วนวันไหนถ้าเกิดอยา กกินbuffetเราจะไปกินตอนปร...
2,แก้คำผิด: ลืมอวดพู่กันของ black velvet(us) รุ...,ลืมอวดพู่กันของ black velvet(us) รุ่น 3100st(#...
3,แก้คำผิด: สนุกนะคะไม่ได้เน้นหื่นอะไรขนาดนั้น ...,สนุกนะคะไม่ได้เน้นหื่นอะไรขนาดนั้น เราว่าฉากพร...
4,แก้คำผิด: สนุกนะคะไม่ได้เน้นหื่นอะไรขนาดนั้น ...,สนุกนะคะไม่ได้เน้นหื่นอะไรขนาดนั้น เราว่าฉากพร...
...,...,...
197172,แก้คำผิด: ตัวพระหน้าตายแบบหน้าหมาซึ่งแม่งมีหล...,ตัวพระหน้าตายแบบหน้าหมาซึ่งแม่งมีหลายเรื่องมาก...
197173,แก้คำผิด: ใจคือแบบไม่มีอะไรหรอกน่า ถ้ามีมันน่...,ใจคือแบบไม่มีอะไรหรอกน่า ถ้ามีมันน่าจะหลังเลือ...
197174,แก้คำผิด: ขออนุญาตแทรก ไม่แน่ใจว่าน้องขยันมาก...,ขออนุญาตแทรก ไม่แน่ใจว่าน้องขยันมาก หรื อว่าผจ...
197175,แก้คำผิด: ขออนุญาตแทรก ไม่แน่ใจว่าน้องขยันมาก...,ขออนุญาตแทรก ไม่แน่ใจว่าน้องขยันมาก หรื อว่าผจ...


In [ ]:
valid_df

,source_text,target_text
0,แก้คำผิด: แค่พูดว่ารักชาติ ศาสน์ กษัตริย์ไม่ไ...,แค่พูดว่ารักชาติ ศาสน์ กษัตริย์ไม่ได้แปลว่าคุณ...
1,แก้คำผิด: อย่างแรกเลยที่ชอบคือเรื่องนี้เป็นฟี...,อย่างแรกเลยที่ชอบคือเรื่องนี้เป็นฟีลกู้ด ที่น่...
2,แก้คำผิด: รีวิวมาม่าอุ่นร้อนอัตโนมัติของจีน ล...,รีวิวมาม่าอุ่นร้อนอัตโนมัติของจีน ลืมถ่ายตอนเอ...
3,แก้คำผิด: รีวิวมาม่าอุ่นร้อนอัตโนมัติของจีน ล...,รีวิวมาม่าอุ่นร้อนอัตโนมัติของจีน ลืมถ่ายตอนเอ...
4,แก้คำผิด: รีวิวมาม่าอุ่นร้อนอัตโนมัติของจีน ล...,รีวิวมาม่าอุ่นร้อนอัตโนมัติของจีน ลืมถ่ายตอนเอ...
...,...,...
21979,แก้คำผิด: สงสารพ่อแม่....เลี้ยงไงเนี่ย ให้ลูก...,สงสารพ่อแม่....เลี้ยงไงเนี่ย ให้ลูกด่าคนอื่นไม...
21980,แก้คำผิด: เมนลิซค่ะ เพราะความเป็นตัวตนของน้อง...,เมนลิซค่ะ เพราะความเป็นตัวตนของน้องและสิ่งที่น...
21981,แก้คำผิด: เมนลิซค่ะ เพราะความเป็นตัวตนของน้อง...,เมนลิซค่ะ เพราะความเป็นตัวตนของน้องและสิ่งที่น...
21982,แก้คำผิด: เมนลิซค่ะ เพราะความเป็นตัวตนของน้อง...,เมนลิซค่ะ เพราะความเป็นตัวตนของน้องและสิ่งที่น...


In [ ]:
test_df

,source_text,target_text
0,แก้คำผิด: จาก ที่ดูvoovของน้อง<mask>คือน้องออ...,จาก ที่ดูvoovของน้อง แล้วคือน้องออกมาแก้ตัวให้...
1,แก้คำผิด: จาก ที่ดูvoovของน้อง แล้วคือน้องออก...,จาก ที่ดูvoovของน้อง แล้วคือน้องออกมาแก้ตัวให้...
2,แก้คำผิด: จาก ที่ดูvoovของน้อง แล้วคือน้องออก...,จาก ที่ดูvoovของน้อง แล้วคือน้องออกมาแก้ตัวให้...
3,แก้คำผิด: ขอพูดถึงสิค<mask> จาก สเตจวันเกิดขอ...,ขอพูดถึงสิคนิดหนึ่ ง จาก สเตจวันเกิดของน้อง ที...
4,แก้คำผิด: ขอพูดถึงสิคนิดหนึ่ ง จาก สเตจวันเกิ...,ขอพูดถึงสิคนิดหนึ่ ง จาก สเตจวันเกิดของน้อง ที...
...,...,...
31539,แก้คำผิด: ระบบakb48เราลงสุ่มกับโอชิที่อยา กถ่...,ระบบakb48เราลงสุ่มกับโอชิที่อยา กถ่ายถ้าสุ่มได...
31540,แก้คำผิด: <mask> เบาๆกับยูนิตแกงส้ม (6) - ห...,อัปเดต เบาๆกับยูนิตแกงส้ม (6) - หลังจาก นี้ ...
31541,แก้คำผิด: อัปเดต<mask> เบาๆกับยูนิตแกงส้ม (6...,อัปเดต เบาๆกับยูนิตแกงส้ม (6) - หลังจาก นี้ ...
31542,แก้คำผิด: อัปเดต <mask> เบาๆกับยูนิตแกงส้ม (6...,อัปเดต เบาๆกับยูนิตแกงส้ม (6) - หลังจาก นี้ ...


In [ ]:
pl.seed_everything(16)
torch.cuda.empty_cache()


class MT5Dataset(torch.utils.data.Dataset): #used for tokenizing
    def __init__(self, df, tokenizer):
        self.data = df.reset_index()
        self.tokenizer = tokenizer
        self.source_max_len = 1024
        self.target_max_len = 1024

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_row = self.data.iloc[idx]
        source, target = data_row["source_text"], data_row["target_text"]

        source_encoding = self.tokenizer(
            source,
            padding="max_length",
            max_length=self.source_max_len,
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        target_encoding = self.tokenizer(
            target,
            padding="max_length",
            max_length=self.target_max_len,
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors="pt",
        )
        #tokenizes input and label

        # Ensure labels are correct (see huggingface T5 training documentation)
        labels = target_encoding.input_ids
        labels[labels == self.tokenizer.pad_token_id] = -100

        return dict(
            input_ids=source_encoding.input_ids.flatten(),
            attention_mask=source_encoding.attention_mask.flatten(),
            decoder_input_ids=labels.flatten(),
            decoder_attention_mask=target_encoding.attention_mask.flatten(),
        )
        #input: source_encoding
        #label (decoder_input): target_encoding


class MT5DataModule(pl.LightningDataModule): #make dataloader via MT5Dataset
    def __init__(
        self,
        tokenizer,
        train_df,
        valid_df,
        test_df,
        batch_size: int = 16,
        num_workers: int = 2,
    ):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.valid_df = valid_df
        self.test_df = test_df
        self.tokenizer = tokenizer

    def setup(self, stage: Optional[str] = None, batch_size=1):
        self.batch_size = batch_size
        if stage == "fit" or stage is None:
            self.train_data = MT5Dataset(self.train_df, self.tokenizer)
            self.valid_data = MT5Dataset(self.valid_df, self.tokenizer)

        if stage == "test" or stage is None:
            self.test_data = MT5Dataset(self.test_df, self.tokenizer)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_data, batch_size=self.batch_size, shuffle=True
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.valid_data, batch_size=self.batch_size, shuffle=False
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_data, batch_size=self.batch_size, shuffle=False
        )


class MT5Lightning(pl.LightningModule):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.avg_training_loss = None
        self.avg_val_loss = None

    def forward(
        self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask
    ):
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        decoder_input_ids = batch["decoder_input_ids"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )

        self.log("loss", output[0], prog_bar=True, on_step=True, on_epoch=True)

        return output[0]

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        decoder_input_ids = batch["decoder_input_ids"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )

        self.log("val_loss", output[0], prog_bar=True, on_step=True, on_epoch=True)

        return output[0]

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        decoder_input_ids = batch["decoder_input_ids"]
        decoder_attention_mask = batch["decoder_attention_mask"]

        output = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
        )

        self.log("test_loss", output.loss, prog_bar=True)

        return output.loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=3e-4)

    def training_epoch_end(self, training_step_outputs):
        self.avg_training_loss = np.round(
            torch.mean(torch.stack([x["loss"] for x in training_step_outputs])).item(),
            4,
        )
        path = ""
        if os.path.exists("drive"):
            path += "drive/MyDrive/mt5-thai-qg/"
        else:
            path += "outputs/"
        path += f"mt5-qg-epoch-{self.current_epoch}-train-loss-{self.avg_training_loss}-val-loss-{self.avg_val_loss}"
        self.tokenizer.save_pretrained(path)
        self.model.save_pretrained(path)

    def validation_epoch_end(self, validation_step_outputs):
        _loss = [x.cpu() for x in validation_step_outputs]
        self.avg_val_loss = np.round(
            torch.mean(torch.stack(_loss)).item(),
            4,
        )


INFO:pytorch_lightning.utilities.seed:Global seed set to 16


In [ ]:
model = MT5ForConditionalGeneration.from_pretrained(
    "google/mt5-small", return_dict=True
)
tokenizer = MT5TokenizerFast.from_pretrained("google/mt5-small")



# model = MT5ForConditionalGeneration.from_pretrained(
#     "drive/MyDrive/mt5-thai-qg/mt5-qg-epoch-8-train-loss-0.0144-val-loss-0.278",
#     return_dict=True,
# )
# tokenizer = MT5TokenizerFast.from_pretrained(
#     "drive/MyDrive/mt5-thai-qg/mt5-qg-epoch-8-train-loss-0.0144-val-loss-0.278"
# )

model.cuda()

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (w

In [ ]:
dataset = MT5DataModule(tokenizer, train_df, valid_df, test_df, batch_size=1)

MT5Model = MT5Lightning(model, tokenizer)

callbacks = []
callbacks.append(EarlyStopping(monitor="val_loss", mode="min"))

# wandb_logger = WandbLogger(
#     project="mT5-thai-multiple-e2e-qg", name="mT5-small-thai-multiple-e2e-qg"
# )

trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    # logger=wandb_logger,
    max_epochs=20,
    log_every_n_steps=1,
    callbacks=callbacks,
    accumulate_grad_batches=20
)

trainer.fit(MT5Model, dataset)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 
------------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:230: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:230: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

In [ ]:
def predict(text):
    with torch.no_grad():
        input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

        input_ids = input_ids.cuda()

        # print(input_ids)
        model.cuda()
        generated_ids = model.generate(
            input_ids=input_ids,
            num_beams=3,
            max_length=10000,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            top_p=50,
            top_k=20,
            num_return_sequences=1,
        )

        # print(generated_ids)

        preds = [
            tokenizer.decode(
                g,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )
            for g in generated_ids
        ]
    return preds


In [ ]:
text_to_predict = """แก้คำผิด: เข้าไม่ได้ <mask><mask>ครับ"""
predicted = predict(text_to_predict)
predicted


['เข้าไม่ได้ อะครับ']

In [ ]:
cou = 0
for i in range(100):
  text_to_predict = test_df.iloc[i]['source_text']
  labels = test_df.iloc[i]['target_text']
  print(f"INPUT: {text_to_predict}")
  print(f"LABEL: {labels}")
  predicted = predict(text_to_predict)
  print(predicted[0])
  if predicted[0] == labels:
    cou += 1
  print("------------------------------")
print(f"{cou} / 20")

INPUT: แก้คำผิด:  จําเป็นต้องเปิดบริการกับสาขาที่เปิดบัญชี <mask>ครับ หรือสาขาไหนก็ได้
LABEL: จําเป็นต้องเปิดบริการกับสาขาที่เปิดบัญชี ไหมครับ หรือสาขาไหนก็ได้
จําเป็นต้องเปิดบริการกับสาขาที่เปิดบัญชี อะครับ หรือสาขาไหนก็ได้
------------------------------
INPUT: แก้คำผิด:  เมื่อวาน<mask>ยังเข้าได้
LABEL: เมื่อวาน ก็ยังเข้าได้
เมื่อวาน ก็ยังเข้าได้
------------------------------
INPUT: แก้คำผิด:  ผม<mask>ต่างประเทศครับ
LABEL: ผมอยู่ต่างประเทศครับ
ผมอยู่ต่างประเทศครับ
------------------------------
INPUT: แก้คำผิด:  พอไปสมัครที่<mask>
LABEL: พอไปสมัครที่ตู้
พอไปสมัครที่ตู้
------------------------------
INPUT: แก้คำผิด:  พอไปสมัครที่ตู้<mask>
LABEL: พอไปสมัครที่ตู้
พอไปสมัครที่ตู้ค่ะ
------------------------------
INPUT: แก้คำผิด:  <mask>มีปัญหาหรือคะตอนนี้
LABEL: แอพมีปัญหาหรือคะตอนนี้
ก็ไม่ได้มีปัญหาหรือคะตอนนี้
------------------------------
INPUT: แก้คำผิด:  แอพมีปัญหา<mask>คะตอนนี้
LABEL: แอพมีปัญหาหรือคะตอนนี้
แอพมีปัญหาหรือเปล่าคะตอนนี้
------------------------------
INPUT: แก้คำผ

In [ ]:
amodel = AutoModelForMaskedLM.from_pretrained("bookpanda/wangchanberta-base-att-spm-uncased-masking")

https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpeyypael4


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

storing https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/8d67990c57ad7c9a377a717b9e6fb6fbe625876c4c9781b591c91d90df72749f.2557f0f732e8aaba8f91e0a2524d25837aa0e1facac5cfefcfdf204769c20fa8
creating metadata file for /root/.cache/huggingface/transformers/8d67990c57ad7c9a377a717b9e6fb6fbe625876c4c9781b591c91d90df72749f.2557f0f732e8aaba8f91e0a2524d25837aa0e1facac5cfefcfdf204769c20fa8
loading configuration file https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8d67990c57ad7c9a377a717b9e6fb6fbe625876c4c9781b591c91d90df72749f.2557f0f732e8aaba8f91e0a2524d25837aa0e1facac5cfefcfdf204769c20fa8
Model config CamembertConfig {
  "_name_or_path": "bookpanda/wangchanberta-base-att-spm-uncased-masking",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_t

Downloading:   0%|          | 0.00/427M [00:00<?, ?B/s]

storing https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/46c75644854dc9aaf27b979f062443e93422a14381fbf8e1f6dcb519ec3ba566.6bf7aed09fc9af5c1beef4b0c4ec485fb5139cae7309b85b7d8e6c9f8bb66fe0
creating metadata file for /root/.cache/huggingface/transformers/46c75644854dc9aaf27b979f062443e93422a14381fbf8e1f6dcb519ec3ba566.6bf7aed09fc9af5c1beef4b0c4ec485fb5139cae7309b85b7d8e6c9f8bb66fe0
loading weights file https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/46c75644854dc9aaf27b979f062443e93422a14381fbf8e1f6dcb519ec3ba566.6bf7aed09fc9af5c1beef4b0c4ec485fb5139cae7309b85b7d8e6c9f8bb66fe0
All model checkpoint weights were used when initializing CamembertForMaskedLM.

All the weights of CamembertForMaskedLM were initialized from the model checkpoint at bookpanda/wangchanberta-base-att-s

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
model.resize_token_embeddings(len(tokenizer))

FILE = "drive/MyDrive/AIBuilders/json/mlm_json_3.pth"
model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

CamembertForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25354, 768)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [ ]:
model

CamembertForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25354, 768)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

# Data Prep

In [ ]:
df = pickle.load(open('drive/MyDrive/AIBuilders/json/mlm_json_15k.pkl', 'rb'))

In [ ]:
df_test = df[10576:]
df = df[:10576]
df_train, df_val = np.split(df.sample(frac=1, random_state=42),
                            [int(.9 * len(df))])

In [ ]:
df_train

,input_ids,attention_mask,labels
1284,"[5, 5248, 51, 25004, 25004, 6, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 5248, 51, 25101, 265, 6, 1, 1, 1, 1, 1, 1,..."
6997,"[5, 10, 1167, 1045, 878, 25004, 25004, 25004, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[5, 10, 1167, 1045, 878, 627, 70, 265, 6, 1, 1..."
9188,"[5, 10, 1145, 1044, 78, 70, 471, 22167, 19404,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 1145, 1044, 78, 70, 471, 22167, 19404,..."
5076,"[5, 10, 1834, 13725, 2102, 25004, 25004, 199, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[5, 10, 1834, 13725, 2102, 234, 265, 199, 1454..."
1393,"[5, 330, 13621, 25004, 25004, 25004, 6, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 330, 13621, 25105, 265, 265, 6, 1, 1, 1, 1..."
...,...,...,...
4757,"[5, 627, 1280, 25004, 25004, 25004, 25004, 250...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 627, 1280, 231, 265, 265, 726, 265, 126, 1..."
10098,"[5, 729, 25004, 1481, 1116, 6, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 729, 8704, 1481, 1116, 6, 1, 1, 1, 1, 1, 1..."
6320,"[5, 7894, 1063, 145, 1849, 300, 25004, 6, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[5, 7894, 1063, 145, 1849, 300, 73, 6, 1, 1, 1..."
5975,"[5, 2169, 25004, 864, 286, 94, 25004, 25004, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 2169, 15, 864, 286, 94, 415, 260, 23, 695,..."


In [ ]:
print(df_train.iloc[0]['input_ids'])
print(df_train.iloc[0]['attention_mask'])
print(df_train.iloc[0]['labels'])

[5, 5248, 51, 25004, 25004, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
df_train = Dataset.from_pandas(df_train)
df_val = Dataset.from_pandas(df_val)
df_test = Dataset.from_pandas(df_test)

In [ ]:
df_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', '__index_level_0__'],
    num_rows: 9518
})

In [ ]:
combined_dataset = DatasetDict({
    'train': df_train,
    'test': df_val,
    'valid': df_test})

In [ ]:
combined_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', '__index_level_0__'],
        num_rows: 9518
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', '__index_level_0__'],
        num_rows: 1058
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

In [ ]:
torch.cuda.empty_cache()
import gc
del df
del df_train
del df_val
del df_test
gc.collect()

461

In [ ]:
from huggingface_hub import notebook_login

notebook_login() # เอาไว้โยนโมเดลขึ้น hugging face ได้เลย

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from transformers import TrainingArguments

batch_size = 16
# Show the training loss with every epoch
logging_steps = len(combined_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-masking",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    num_train_epochs = 3,
    fp16=True, # สำหรับคนใช้ GPU
    logging_steps=logging_steps,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=combined_dataset["train"],
    eval_dataset=combined_dataset["valid"],
)

Cloning https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking into local empty directory.


Download file pytorch_model.bin:   0%|          | 83.0/427M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 2.92k/2.92k [00:00<?, ?B/s]

Clean file training_args.bin:  34%|###4      | 1.00k/2.92k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/427M [00:00<?, ?B/s]

Using amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CamembertForMaskedLM.forward` and have been ignored: __index_level_0__.
***** Running training *****
  Num examples = 9518
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1785


Epoch,Training Loss,Validation Loss
1,0.019100,0.020692
2,0.019100,0.020692
3,0.019000,0.020692


Saving model checkpoint to wangchanberta-base-att-spm-uncased-masking/checkpoint-500
Configuration saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-500/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to wangchanberta-base-att-spm-uncased-masking/checkpoint-1000
Configuration saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-1000/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to wangchanberta-base-att-spm-uncased-masking/checkpoint-1500
Configuration saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-1500/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-masking/checkpoint-1500/pytorch_model.bin
***** Running Evaluat

TrainOutput(global_step=1785, training_loss=0.01904480025506153, metrics={'train_runtime': 2196.4521, 'train_samples_per_second': 13.0, 'train_steps_per_second': 0.813, 'total_flos': 7515096898646016.0, 'train_loss': 0.01904480025506153, 'epoch': 3.0})

In [ ]:
FILE = "/content/drive/MyDrive/AIBuilders/json/mlm_json_6.pth"
torch.save(model.state_dict(), FILE)

In [ ]:
trainer.push_to_hub()# โยนขึ้น hugging face

Saving model checkpoint to wangchanberta-base-att-spm-uncased-masking
Configuration saved in wangchanberta-base-att-spm-uncased-masking/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-masking/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.34k/427M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 2.92k/2.92k [00:00<?, ?B/s]

To https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking
   0ad3488..042aaa7  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Masked Language Modeling', 'type': 'fill-mask'}}
To https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking
   042aaa7..d124223  main -> main



'https://huggingface.co/bookpanda/wangchanberta-base-att-spm-uncased-masking/commit/042aaa7a82de0ea8414ca4f8eb1e486c59978c17'

In [ ]:
class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if use_cuda:
    model = model.cuda()
FILE = "drive/MyDrive/AIBuilders/mlm/tagging_nova_75.pth"
loaded_model = BertModel()
loaded_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
loaded_model.eval()

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.3.attention.output.dense.weight', 'roberta.encoder.layer.9.attention.self.key.weight', 'lm_head.bias', 'roberta.encoder.layer.1.attention.self.value.bias', 'roberta.encoder.layer.10.attention.self.value.bias', 'roberta.encoder.layer.7.attention.self.query.weight', 'roberta.encoder.layer.11.output.LayerNorm.weight', 'roberta.encoder.layer.10.output.LayerNorm.weight', 'roberta.encoder.layer.2.intermediate.dense.weight', 'roberta.encoder.layer.8.attention.self.value.bias', 'roberta.encoder.layer.6.attention.output.LayerNorm.bias', 'roberta.encoder.layer.7.attention.self.key.bias', 'lm_head.dense.bias', 'roberta.encoder.layer.11.attention.self.query.weight', 'roberta.enco

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(33660, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [ ]:
ids_to_labels = {0: 'f', 1: 'i'}

def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)
    c = tokenizer.convert_ids_to_tokens(tokenized_inputs.input_ids)
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)
        else:
            try:
              label_ids.append(2)
            except:
                label_ids.append(-100)

        previous_word_idx = word_idx
    return label_ids

def evaluate_one_text(model, sentence):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'][0].unsqueeze(0).to(device)

    input_id = text['input_ids'][0].unsqueeze(0).to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
# if use_cuda:
#     model = model.cuda()

text = "ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ"
ans = []
i_f = evaluate_one_text(loaded_model, text)
print(i_f)
a = tokenizer(text)
b = a['input_ids']
c = tokenizer.convert_ids_to_tokens(b)
print(c)
i_f_len = len(i_f)
for j in range(i_f_len):
  if(i_f[j] == 'i'):
    ph = a['input_ids'][j+1]
    a['input_ids'][j+1] = 25004
    print(tokenizer.decode(a['input_ids']))
    b = {'input_ids': torch.Tensor([a['input_ids']]).type(torch.int64).to(device), 'attention_mask': torch.Tensor([a['attention_mask']]).type(torch.int64).to(device)}
    token_logits = model(**b).logits
    mask_token_index = torch.where(b["input_ids"] == tokenizer.mask_token_id)[1]
    mask_token_logits = token_logits[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
    ans.append((j, top_5_tokens[0]))
    text = ''.join(tokenizer.convert_ids_to_tokens(a['input_ids']))
    for token in top_5_tokens:
        print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")
    a['input_ids'][j+1] = ph

print(a)
for x,y in ans:
  a['input_ids'][x+1] = y
print(''.join(tokenizer.convert_ids_to_tokens(a['input_ids'])))

['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'i', 'i', 'i']
['<s>', '▁', 'ประเทศ', 'เรา', 'ผลิต', 'และ', 'ส่งออก', 'ยา', '▁', 'สูบ', 'เยอะ', 'สุดในโลก', 'จิง', 'ป่าว', 'คับ', '</s>']
<s> ประเทศเราผลิตและส่งออกยา สูบเยอะสุดในโลก<mask>ป่าวคับ</s>
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกหรือป่าวคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกอะป่าวคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกหรื อป่าวคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลก จริง ๆป่าวคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกอ๊ะป่าวคับ</s>'
<s> ประเทศเราผลิตและส่งออกยา สูบเยอะสุดในโลกจิง<mask>คับ</s>
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิง<pad>คับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิงไหมคับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิง<s>คับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิง_คับ</s>'
'>>> <s>▁ประเทศเราผลิตและส่งออกยา▁สูบเยอะสุดในโลกจิงอะคับ</s>'
<s> ประเทศเราผลิตและส่งออกยา สูบเ

In [ ]:
asdsf = pd.read_csv("drive/MyDrive/AIBuilders/final.csv")

In [ ]:
for i in range(100,200):
  print(asdsf.iloc[i]['text'])

ต้องลอง555
อยากได้เตาอบบาบีก้อน จะเอามาทำขนมให้ลูกๆ ไปทานเล่นระหว่างวันค่ะ เป็นแม่บ้านว่างๆ ก็จะสรรหาเมนูใหม่ๆ ตามยูทูปเอามาลองทำ มีลูกนี่แหละเป็นหนูทดลอง 555 เห็นเตาก้อนแล้วอยากทำขนมคุ๊กกี้อบมากเลยค่ะ เอาเป็นรูปซานต้ากับต้นคริสมาส ฝึกทันพอดีก่อนเทศกาลเลย ❤️❤️❤️❤️❤️
เกิน 50 ไม่ไป!
แฟชั่นไอส์แลนด์ วันที่3/12ค่า
ลดพิเศษเเต่ของน้อยไม่ค่อยเติม
ฮื้ออออ เค้าสับเพร่า เค้าสับเพร่าาาาา เค้าจ่ายเต็มมมมม แต่อร่อยเด้ออออ ฮื้อออ
ผ่านนิเทศไปก่อนนะ
ไปคนเดียวดีฝ่าา
สาขาเพลินนารี่ ถ้าไม่อยากขายก้อปิดเถอะคับ ดูแลไม่ดีแล้วพนง.ยังไม่มีจิตใจในการขายด้วย เวลา1ชม.กับหลายๆอย่างไม่ไหวคับ หาคนมาดูบ้างนะครับ
ขาดเลยละ55 ไฮเนเก้น พอๆๆ 2 ขวด 555
ลอรีอัล รีไวทัลลิฟท์ เดอร์มาลิฟท์เดย์ครีม 50กรัม/ไนท์ ครีม 50กรัม ได้รับคูปองส่วนลด จากราคาเต็ม 599 บาท ลด 120บาท เหลือ 479บาท ในโปรโมชั่น watsons Beauty in Summer (29 มี.ค.-25 เม.ย. 61)
แล้วโครงการอื่นล่ะ??? แสนสิริเตรียมแจงกรณีปัญหาผนังห้องชุดโครงการเดอะ เบส สุขุมวิท 77 พรุ่งนี้ หลังพบใช้โฟมเป็นวัสดุก่อผนัง”
มีซูชิหรอ
นิสสันโน๊ตครับ
ขอบพระคุณคร้าบ
ขอสำรองได้มั้ยอ่า
ตอนนอน